# Predcition

We first try to detect P waves which are the first waves to arrive at a seismic station and  use the real-time  time series data to make a prediction about the earthquake by calculating the time taken for other waves to arrive after P waves are detected. We assume the data to be stored in a dataset called time-series .


In [0]:
import numpy as np
from pandas import Series, DataFrame
from pandas.tseries.offsets import Hour,Minute
from datetime import datetime
import pytz

ts1 = pd.read_csv(' /time series/') #Loading the time series data for the P waves
ts2 = pd.read_csv('/time series/') #Loading the time series data for the S waves
minute = Minute()
ts1_utc = ts.tz_localize('UTC') #Conversion from naive to localized time
ts2_utc = ts.tz_localizr('UTC')                 
                 
pd.date_range('date1' , 'date2', freq = '1m') #Getting values every minute

ts = ts2 - ts1



                 

In [0]:
#Applying Random Forest to predict the time remaining before the S waves arrive:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
X_train,X_test,y_train,y_test = train_test_split(ts.time,ts.values) #Splitting the data into training and test sets
lreg = LinearRegression()
forest = RandomForestRegressor(random_state=0)
forest.fit(X_train,y_train)

print("Accuracy on the training set: %f" %forest.score(X_train,y_train))
print("Accuracy on the test set: %f" %forest.score(X_test,y_test))

coeff = DataFrame(x_train.columns)

coeff['Coefficient Estimate'] = Series(lreg.coef_)

coeff




# Relief


In [0]:
#We will collect the dataset of images of people residing within an area
#Then train our model on it to predict who all are missing in case of an earthquake

import numpy as np
import numpy.random as rng
import os
from matplotlib.pyplot as plt
import dill as pickle
from sklearn.utils import shuffle
import tensorflow as tf
from keras.layers import Input, Conv2D, Lambda, merge, Dense, Flatten,MaxPooling2D
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K
from keras.optimizers import SGD,Adam
from keras.losses import binary_crossentropy


In [0]:
def W_init(shape, name=None):
  values = rng.normal(loc=0,scale=1e-2,size=shape)
    return K.variable(values,name=name)
def b_init(shape, name=None):
  values=rng.normal(loc=0.5,scale=1e-2,size=shape)
    return K.variable(values,name=name)

input_shape = (value,value,value)
left_input = Input(input_shape)
right_input = Input(input_shape)
#build convnet to use in each siamese 'leg'
convnet = Sequential()
convnet.add(Conv2D(64,(10,10),activation='relu',input_shape=input_shape,
                   kernel_initializer=W_init,kernel_regularizer=l2(2e-4)))
convnet.add(MaxPooling2D())
convnet.add(Conv2D(128,(7,7),activation='relu',
                   kernel_regularizer=l2(2e-4),kernel_initializer=W_init,bias_initializer=b_init))
convnet.add(MaxPooling2D())
convnet.add(Conv2D(128,(4,4),activation='relu',kernel_initializer=W_init,kernel_regularizer=l2(2e-4),bias_initializer=b_init))
convnet.add(MaxPooling2D())
convnet.add(Conv2D(256,(4,4),activation='relu',kernel_initializer=W_init,kernel_regularizer=l2(2e-4),bias_initializer=b_init))
convnet.add(Flatten())
convnet.add(Dense(4096,activation="sigmoid",kernel_regularizer=l2(1e-3),kernel_initializer=W_init,bias_initializer=b_init))
#encode each of the two inputs into a vector with the convnet
encoded_l = convnet(left_input)
encoded_r = convnet(right_input)
#merge two encoded inputs with the l1 distance between them
L1_distance = lambda x: K.abs(x[0]-x[1])
both = merge([encoded_l,encoded_r], mode = L1_distance, output_shape=lambda x: x[0])
prediction = Dense(1,activation='sigmoid',bias_initializer=b_init)(both)
siamese_net = Model(input=[left_input,right_input],output=prediction)
#optimizer = SGD(0.0004,momentum=0.6,nesterov=True,decay=0.0003)

optimizer = Adam(0.00006)
siamese_net.compile(loss="binary_crossentropy",optimizer=optimizer)

siamese_net.count_params()

In [0]:

Siamese_Loader:
    """For loading batches and testing tasks to a siamese net"""
    def __init__(self,Xtrain,Xval):
        self.Xval = Xval
        self.Xtrain = Xtrain
        self.n_classes,self.n_examples,self.w,self.h = Xtrain.shape
        self.n_val,self.n_ex_val,_,_ = Xval.shape

    def get_batch(self,n):
        """Create batch of n pairs, half same class, half different class"""
        categories = rng.choice(self.n_classes,size=(n,),replace=False)
        pairs=[np.zeros((n, self.h, self.w,1)) for i in range(2)]
        targets=np.zeros((n,))
        targets[n//2:] = 1
        for i in range(n):
            category = categories[i]
            idx_1 = rng.randint(0,self.n_examples)
            pairs[0][i,:,:,:] = self.Xtrain[category,idx_1].reshape(self.w,self.h,1)
            idx_2 = rng.randint(0,self.n_examples)
            #pick images of same class for 1st half, different for 2nd
            category_2 = category if i >= n//2 else (category + rng.randint(1,self.n_classes)) % self.n_classes
            pairs[1][i,:,:,:] = self.Xtrain[category_2,idx_2].reshape(self.w,self.h,1)
        return pairs, targets

    def make_oneshot_task(self,N):
        """Create pairs of test image, support set for testing N way one-shot learning. """
        categories = rng.choice(self.n_val,size=(N,),replace=False)
        indices = rng.randint(0,self.n_ex_val,size=(N,))
        true_category = categories[0]
        ex1, ex2 = rng.choice(self.n_examples,replace=False,size=(2,))
        test_image = np.asarray([self.Xval[true_category,ex1,:,:]]*N).reshape(N,self.w,self.h,1)
        support_set = self.Xval[categories,indices,:,:]
        support_set[0,:,:] = self.Xval[true_category,ex2]
        support_set = support_set.reshape(N,self.w,self.h,1)
        pairs = [test_image,support_set]
        targets = np.zeros((N,))
        targets[0] = 1
        return pairs, targets

    def test_oneshot(self,model,N,k,verbose=0):
        """Test average N way oneshot learning accuracy of a siamese neural net over k one-shot tasks"""
        pass
        n_correct = 0
        if verbose:
            print("Evaluating model on {} unique {} way one-shot learning tasks ...".format(k,N))
        for i in range(k):
            inputs, targets = self.make_oneshot_task(N)
            probs = model.predict(inputs)
            if np.argmax(probs) == 0:
                n_correct+=1
        percent_correct = (100.0*n_correct / k)
        if verbose:
            print("Got an average of {}% {} way one-shot learning accuracy".format(percent_correct,N))
        return percent_correct

evaluate_every = 7000
loss_every=300
batch_size = 32
N_way = 20
n_val = 550
siamese_net.load_weights("PATH")
best = 76.0
for i in range(900000):
    (inputs,targets)=loader.get_batch(batch_size)
    loss=siamese_net.train_on_batch(inputs,targets)
    if i % evaluate_every == 0:
        val_acc = loader.test_oneshot(siamese_net,N_way,n_val,verbose=True)
        if val_acc >= best:
            print("saving")
            siamese_net.save('PATH')
            best=val_acc

    if i % loss_every == 0:
        print("iteration {}, training loss: {:.2f},".format(i,loss))


# Analysis


We will be monitoring the real time seismic waves being emitted and use Neural Network to calculate the magnitude of earthquake

In [0]:
#We train our model using 'Earthquake-Normalized' dataset.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [0]:
df = pd.read_csv('Earthquake_normalized.csv')
col1 = df['DateTime','Latitude','Longitude','Depth']
col2 = df['Magnitude']

x = col1.as_matrix()
y = col2.as_matrix()

#Reshape
x_features = 4 #Number of input features
y_features = 1 #Number of input features
samples = 23000 # Number of samples

Inputx1_reshape = np.resize(x1,(samples,x_features))
Inputy1_reshape = np.resize(y1,(samples,y_features))

#Training data
batch_size = 20000
Inputx1_train = Inputx1_reshape[0:batch_size,:]
Inputy1_train = Inputy1_reshape[0:batch_size,:]
#Validation data
val_size = 2500
Inputx_1val = Inputx1_reshape[batch_size:batch_size+val_size,:]
Inputy_1val = Inputy1_reshape[batch_size:batch_size+val_size,:]
#print(Inputx_1val)


#Hyper parameters:
learning_rate = 0.0005
training_iterations = 100000

#Variables:
x = tf.placeholder((tf.float32,shape=(None,x_features))
y = tf.placeholder(tf.float32)
                   
#Neurons
#L1 = Number of neurons in 1st layer
#L2 = Number of neurons in 2nd layer
#L3 = Number of neurons in 3rd layer
#Layer1 weights
W_fc1 = tf.Variable(tf.random_uniform([Xfeatures,L1])) # [input_features,Number of neurons]) 
b_fc1 = tf.Variable(tf.constant(0.1,shape=[L1]))
#Layer2 weights
#W_fc2 = tf.Variable(tf.random_uniform([L1,L2])) # [Number of neurons in preceding layer,Number of neurons]) 
#b_fc2 = tf.Variable(tf.constant(0.1,shape=[L2]))
#Layer3 weights
#W_fc3 = tf.Variable(tf.random_uniform([L1,L3])) # [Number of neurons in preceding layer,Number of neurons]) 
#b_fc3 = tf.Variable(tf.constant(0.1,shape=[L3]))
#Output layer weights
W_fO= tf.Variable(tf.random_uniform([L1,Yfeatures])) #  [Number of neurons in preceding layer,output_features]) 
b_fO = tf.Variable(tf.constant(0.1,shape=[Yfeatures]))    
                   
#Layer 1
matmul_fc1=tf.matmul(X, W_fc1) + b_fc1
h_fc1 = tf.nn.relu(matmul_fc1)   #ReLU activation
#Layer 2
#matmul_fc2=tf.matmul(h_fc1, W_fc2) + b_fc2
#h_fc2 = tf.nn.relu(matmul_fc2)   #ReLU activation
#Layer 3
#matmul_fc3=tf.matmul(h_fc1, W_fc3) + b_fc3
#h_fc3 = tf.nn.relu(matmul_fc3)   #ReLU activation
#Output layer
matmul_fc4=tf.matmul(h_fc1, W_fO) + b_fO
output_layer = matmul_fc4  #linear activation     
                   
#Loss function
mean_square =  tf.reduce_mean(tf.square(y_output_layer))
train_step = tf.train.AdamOptimizer(learning_rate).minimize(mean_square)

#Operation to save variables
saver = tf.train.Saver()                   
        
#Initialization and session:
init = tf.initialize_all_variables()
with tf.Session() as sess:
    sess.run(init)
    print("Training loss:",sess.run([mean_square],feed_dict={X:InputX1train,Y:InputY1train}))
    for i in range(training_iterations):
        sess.run([train_step],feed_dict={X:InputX1train,Y:InputY1train})
        if i%display_iterations ==0:
            print("Training loss is:",sess.run([mean_square],feed_dict={X:InputX1train,Y:InputY1train}),"at itertion:",i)
            print("Validation loss is:",sess.run([mean_square],feed_dict={X:InputX1v,Y:InputY1v}),"at itertion:",i)
    # Save the variables to disk.
    save_path = saver.save(sess, "/tmp/earthquake_model.ckpt")
    print("Model saved in file: %s" % save_path)

    print("Final training loss:",sess.run([mean_square],feed_dict={X:InputX1train,Y:InputY1train}))
    print("Final validation loss:",sess.run([mean_square],feed_dict={X:InputX1v,Y:InputY1v}))                   